In [ ]:
from datascience import *
import numpy as np

## Maps

In [ ]:
stations = Table.read_table('station.csv')
stations

In [ ]:
Marker.map_table(stations.select('lat', 'long', 'name'))

In [ ]:
Circle.map_table(stations.select('lat', 'long', 'name'), radius=150, color='green')

In [ ]:
trip = Table.read_table('trip.csv').where('Duration', are.below(1800)).select(3, 6, 1).relabeled(0, 'Start').relabeled(1, 'End')

In [ ]:
starts = trip.group('Start').sort('count', descending=True)
starts

In [ ]:
station_starts = stations.join('name', starts, 'Start')
station_starts

In [ ]:
landmarks = stations.group('landmark')
landmarks

In [ ]:
landmarks = landmarks.with_column(
    'color',
    make_array('blue', 'red', 'yellow', 'orange', 'purple')
)
landmarks

In [ ]:
station_starts = station_starts.join('landmark', landmarks.drop('count'))
station_starts

In [ ]:
station_starts = station_starts.with_column(
    'circle size',
    station_starts.column('count') * 1000
)
station_starts

In [ ]:
Circle.map_table(station_starts.select('lat', 'long', 'name', 'color', 'circle size'))

## Table examples

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price']).with_rows([
    ['Milk Tea', 'Tea One', 4],
    ['Espresso', 'Nefeli',  2],
    ['Coffee',   'Nefeli',  3],
    ['Espresso', "Abe's",   2]
])
drinks

In [ ]:
discounts = Table().with_columns(
    'Coupon % off', make_array(5, 50, 25),
    'Location', make_array('Tea One', 'Nefeli', 'Tea One')
)
discounts

In [ ]:
# Link (join) drinks with discounts
combined = drinks.join('Cafe', discounts, 'Location')

# Compute discounted prices
discounted_prices = combined.column('Price') * (1 - combined.column('Coupon % off')/100)

discounted_drinks = combined.with_column(
    'Discounted price', discounted_prices
)
discounted_drinks
# Sort

In [ ]:
discounted_drinks.sort('Discounted price').sort('Cafe', distinct=True) # Correct, Espresso is cheaper

In [ ]:
# Incorrect, Coffee is first alphabetically
discounted_drinks.group('Cafe', min) 

## Spring 2016 Midterm, Question 2(b)

Challenge yourself and try to solve these on your own before looking at the solutions!

In [ ]:
trip.show(3)

In [ ]:
# The name of the station where the most rentals ended (assume no ties).

In [ ]:
# The number of stations for which the average duration ending at that station was more than 10 minutes.

In [ ]:
# The number of stations that have more than 500 starts AND more than 500 ends

In [ ]:
# The name of the station where the most rentals ended (assume no ties).
# First, find end counts
# Then, find the station with the highest end count
trip.group('End').sort('count', descending=True).column(0).item(0)

In [ ]:
# The number of stations for which the average duration ending at that station was more than 10 minutes.

# First, find the average end time for each station
# Then, keep the ones above 10 minutes
# Then, count them
trip.group('End', np.average).where(2, are.above(10*60)).num_rows


In [ ]:
# The number of stations that have more than 500 starts AND more than 500 ends
# First, find the start counts
starting = trip.group('Start').relabeled('count', 'Start count').relabeled('Start', 'Station')
# Then, find the end counts
ending = trip.group('End').relabeled('count', 'End count').relabeled('End', 'Station')
# Combine them with join
starting.join('Station', ending).where('Start count', are.above(500)).where('End count', are.above(500)).num_rows